Having scraped data from IGN and Steam using Selenium I now have to clean the data. 

This will include modifying incorrect values, looking for null values, looking for duplicate rows, and formatting the date.  

First I load my data into a pandas dataframe:

In [ ]:
import pandas as pd

game_df = pd.read_csv('game_df.csv')
game_df.drop('Unnamed: 0', inplace = True, axis=1) #drop extra index column
game_df.tail(10)

From visual inspection of the CSV file I can see that some of the data in the player sentiment column have incorrect spelling. Specifically they have and extra '*' symbol.  

Using the pandas unique method I can see there are also two other incorrect values: 'No user reviews' and '8 user reviews'.

In [ ]:
# use loc to identify cells based on condition that need to be changed
import numpy as np
game_df.loc[ game_df['Player Sentiment'] == 'Overwhelmingly Positive *', 'Player Sentiment'] = 'Overwhelmingly Positive'
game_df.loc[ game_df['Player Sentiment'] == 'Mostly Positive *', 'Player Sentiment'] = 'Mostly Positive'
game_df.loc[ game_df['Player Sentiment'] == 'Very Positive *', 'Player Sentiment'] = 'Very Positive'
game_df.loc[ game_df['Player Sentiment'] == 'Mixed *', 'Player Sentiment'] = 'Mixed'

game_df.loc[ game_df['Player Sentiment'] == 'No user reviews', 'Player Sentiment'] = np.nan
game_df.loc[ game_df['Player Sentiment'] == '8 user reviews', 'Player Sentiment'] = np.nan

game_df['Player Sentiment'].value_counts() # make sure all values are allowed

Next I want to look for missing values. 

I did this by going column-by-column and finding null values. Then manually fixing the missing data at the corresponding indices.

I kept a record of each manual change that I made.

In [ ]:
# find no. of null values
game_df['Price'].isnull().sum()
# get the indices of the null values
game_df[game_df['Price'].isnull()].index
# replace null values
game_df.loc[379 ,'Price'] = 0.0

#game_df.drop(356, inplace = True) #drop row is nessesary
#game_df.reset_index(drop = True, inplace = True) #reset dataframe index

game_df.isnull().sum() #finally check all null values were corrected

I had to drop some games because despite being reviewed by IGN and listed on Steam they are not available for purchase yet.

A majority of missing values were missing price data. These games are free to play on Steam so I just set their price to 0.0.

In [ ]:
#look for duplicate rows
bool_df = game_df.duplicated(subset = ['Positive Reviews [%]','No. of Reviews'], keep = False)
game_df[bool_df == True] #display duplicates
# 6 duplicate rows including the firsts.

#remove duplicates
game_df.drop_duplicates(subset = ['Positive Reviews [%]','No. of Reviews'], keep = 'first', inplace = True)
bool_df = game_df.duplicated(subset = ['Positive Reviews [%]','No. of Reviews'], keep = False)
game_df[bool_df == True]
#All duplicates, apart from the first, have been dropped.

In [ ]:
# convert from string to date time
game_df['Release Date'] = game_df['Release Date'].astype('datetime64[ns]')